# Accounting Analytics  Lab 4-4: Linking Tables in Excel using the Data Model

**Keywords:** Data Model, Linking Tables, Primary Key, Foreign Key, Aggregtion, Pivot Tables

**Insight:** Many types of tabular data exist in separate, normalized sources and need to be combined for analysis. In this lab we will learn how to use database and SQL-like concepts to link and aggregate data from multiple tables.

In [1]:
# Import key library functions needed.
import pandas as pd
import numpy as np

## Part 1: Data Reading and Initial Exploration

#### Step 1: Open the Excel File Lab_5_1_Data.xlsx and read in the tables:

In [2]:
# Specify the excel file containing multiple worksheets (i.e. tables).
data_file = './Lab_4_4_Data.xlsx'

# Create a variable for sales transactions containing the Sales_Transactions worksheet data.
sales_transactions = pd.read_excel(data_file, 'Sales_Transactions')

# Do the same for customers.
customers = pd.read_excel(data_file, 'Customers')

# Do the same for products.
products = pd.read_excel(data_file, 'Products')

#### Step 2: View the First Few Rows of Each Table:

In [3]:
sales_transactions.head()

,Sales_Order_ID,Sales_Order_Date,Sales_Employee_ID,Customer_ID,Product_Code,Sales_Order_Quantity_Sold,Product_Sale_Price
0,20062,2019-11-22,1007,2001,2004,12,105
1,20168,2019-12-27,1007,2001,2005,8,85
2,20383,2020-03-04,1007,2001,2004,5,105
3,20564,2020-04-28,1007,2001,2002,6,120
4,20140,2019-12-20,1006,2002,2001,5,95


In [4]:
customers.head()

,Customer_ID,Business_Name,Customer_Address,Customer_City,Customer_St,Customer_Zip
0,2001,Beverage Distributors,3221 SE 14th Street,Des Moines,IA,50320
1,2002,Deep Ellum Brewing Company,2823 St Louis Street,Dallas,TX,75226
2,2003,Schatz Distributing Co.,3140 S. 28th Steet,Kansas City,KS,66106
3,2004,Arkansas Craft Distributors,1515 E. 4th Street,Little Rock,AR,72202
4,2005,Fleck Sales Co.,11125 High Life Ct SW,Cedar Rapids,IA,52404


In [5]:
products.head()

,Product_Code,Product_Description,Product_Sale_Price
0,2001,Pale Ale,95
1,2002,Imperial IPA,120
2,2003,IPA,100
3,2004,Imperial Stout,105
4,2005,Stout,85


## Part 2: Questions and Exercises

**Queston 1:** What is the Foreign Key in the Sales_Transactions table that allows you to relate it to the Customers table?

*Answer:* Using the tables above, the Customer_ID attribute is found in the Customers table to uniquely identify each customer. The same attribute is found in the Sales_Transactions table to specify the unique customer involved in the transaction.

**Question 2:** What is the Foreign Key in the Sales_Transactions table that allows you to relate it to the Products table?

*Answer:* Similar to above, the Product_Code attribute is found in the Products table to uniquely identify each product. The same attribute is found in the Sales_Transactions table to specify the unique product involved in the transaction.

**Question 3:** How many Imperial Stouts were sold to customers in Arkansas?

In [6]:
# Step 1: Merge the customers with the sales transactions into a new table.
cust_trans_table = customers.merge(sales_transactions, left_on='Customer_ID', right_on='Customer_ID')

# Step 2: Get a single-element product table with product description 'Imperial Stout'.
imperial_stout_table = products[(products['Product_Description'] == 'Imperial Stout')]

# Step 3: Merge imperial_stout_table with cust_trans_table by matching Product Code into a new table.
imp_trans_table = imperial_stout_table.merge(cust_trans_table, left_on='Product_Code', right_on='Product_Code')

# Step 4: Group by state and aggregate using the count and sum to see number of rows and total qty sold.
imp_trans_table.groupby('Customer_St').agg({'Sales_Order_Quantity_Sold': ['sum', 'count']})

Sales_Order_Quantity_Sold      
                                  sum count
Customer_St                                
AR                                 45     6
IA                                 42     6
KS                                 21     3
MO                                 37     6
NE                                 11     2
OK                                 44     6
TX                                 58    12

*Answer:* Looking at the table above, there were 6 transactions with a total quantity of 45 sold to customers in Arkansas.

**Question 4:** What was the total quantity sold to customers from Iowa (for any products)?

In [7]:
# Step 1: Merge the Iowa customer table with with the sales transactions into a new table.
cust_trans_table = customers.merge(sales_transactions, left_on='Customer_ID', right_on='Customer_ID')

# Step 2: Group by customer state and aggregate sales order quantity sold by sum (to get total qty) and 
#         count (to get number of transactional records).
cust_trans_table.groupby('Customer_St').agg({'Sales_Order_Quantity_Sold': ['count', 'sum']})

Sales_Order_Quantity_Sold     
                                count  sum
Customer_St                               
AR                                 20  144
IA                                 11   83
KS                                 14   98
LA                                  2   20
LA                                  2   15
MO                                 20  135
NE                                 16   96
OK                                 24  134
TN                                  5   43
TX                                 35  211

*Answer:* A total qty of 83 products was sold to Iowa customers in 11 transactions.

**Question 5:** Change the Pivot Table so that you see the a count of sales orders (not sum of quantity sold) for each product and state.  How many sales orders with Pale Ale on them were for customers from Kansas?

*Remark:* We build a pivot table grouping by product description followed by customer state. Then we aggregate using both sum (to see the total qty) and count (to see the number of sales orders/transactions).

In [8]:
# Step 1: Merge the customers with the sales transactions into a new table.
cust_trans_table = customers.merge(sales_transactions, left_on='Customer_ID', right_on='Customer_ID')

# Step 2: Merge the products in with cust_trans_table into a new table
cpt_table = products.merge(cust_trans_table, left_on='Product_Code', right_on='Product_Code')

# Step 2: Group by state and aggregate using the count and sum to see number of rows and total qty sold.
cpt_table.groupby(['Product_Description', 'Customer_St']).agg({'Sales_Order_Quantity_Sold': ['sum', 'count']})

Sales_Order_Quantity_Sold      
                                                      sum count
Product_Description Customer_St                                
IPA                 AR                                 50     7
                    KS                                 33     5
                    LA                                 20     2
                    LA                                  3     1
                    MO                                 22     4
                    NE                                 20     4
                    OK                                 28     6
                    TN                                 16     2
                    TX                                 43     7
Imperial IPA        IA                                  6     1
                    KS                                  1     1
                    MO                                 10     2
                    NE                                 12     2
                    OK                                 11     2
                    TN                                 10     1
                    TX                                 11     1
Imperial Stout      AR                                 45     6
                    IA                                 42     6
                    KS                                 21     3
                    MO                                 37     6
                    NE                                 11     2
                    OK                                 44     6
                    TX                                 58    12
Pale Ale            AR                                 14     2
                    IA                                  8     1
                    KS                                 10     1
                    MO                                 13     2
                    NE                                  7     1
                    OK                                 27     4
                    TN                                  8     1
                    TX                                 29     6
Stout               AR                                 31     4
                    IA                                 27     3
                    KS                                 18     2
                    MO                                 33     4
                    NE                                 19     3
                    OK                                 19     4
                    TX                                 43     6
Wheat               AR                                  4     1
                    KS                                 15     2
                    LA                                 12     1
                    MO                                 20     2
                    NE                                 27     4
                    OK                                  5     2
                    TN                                  9     1
                    TX                                 27     3

*Answer:* By looking at the table above, there was one sales order for Pale Ale in Kansas for a qty of 10.